In [ ]:
!pip install conllu
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3105/ud-treebanks-v2.5.tgz
!wget https://www.ida.liu.se/~TDDE09/commons/projectivize.py
!tar zxf ud-treebanks-v2.5.tgz

import os
import torch
import projectivize
import torch.nn as nn
from io import open
from conllu import parse_incr
import copy
import operator
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import subprocess

--2021-03-18 17:15:53--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3105/ud-treebanks-v2.5.tgz
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 355216681 (339M) [application/x-gzip]
Saving to: ‘ud-treebanks-v2.5.tgz’

ud-treebanks-v2.5.t 100%[===================>] 338.76M  18.9MB/s    in 20s     

2021-03-18 17:16:15 (17.1 MB/s) - ‘ud-treebanks-v2.5.tgz’ saved [355216681/355216681]

--2021-03-18 17:16:15--  https://www.ida.liu.se/~TDDE09/commons/projectivize.py
Resolving www.ida.liu.se (www.ida.liu.se)... 130.236.57.103, 2001:6b0:17:2004::57:103
Connecting to www.ida.liu.se (www.ida.liu.se)|130.236.57.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2878 (2.8K)
Saving to: ‘projectivize.py’

projectivize.py     100%[===================>]   2.81K  --.-KB/s    in 0s      

20

In [ ]:
 device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 print(device)

cpu


In [ ]:
#polish train data = UD_Polish-PDB/pl_pdb-ud-train.conllu
#english train data = UD_English-GUM/en_gum-ud-train.conllu

#arabic train data = UD_Arabic-PADT/ar_padt-ud-train.conllu
#arabic dev data = UD_Arabic-PADT/ar_padt-ud-dev.conllu

def datafiles(filename):
  data_file = open(filename, "r", encoding="utf-8")
  ud_files = []
  for tokenlist in parse_incr(data_file):
    ud_files.append(tokenlist)
  data = []
  for sentence in ud_files:
    current_sentence = []
    for token in sentence:
      current_token = []
      current_token.append(token['form'])
      current_token.append(token['upostag'])
      current_sentence.append(tuple(current_token))
    data.append(current_sentence)
  return data

#Arabic
arabic_train_data = datafiles("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-train.conllu")
arabic_dev_data = datafiles("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-dev.conllu")
#English
english_train_data = datafiles("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-train.conllu")
english_dev_data = datafiles("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-dev.conllu")

In [ ]:
print("Data Examples")
print(english_train_data[0])
print(arabic_train_data[0])
#We are not really using this dataset so move on haha

Data Examples
[('Aesthetic', 'ADJ'), ('Appreciation', 'NOUN'), ('and', 'CCONJ'), ('Spanish', 'ADJ'), ('Art', 'NOUN'), (':', 'PUNCT')]
[('برلين', 'X'), ('ترفض', 'VERB'), ('حصول', 'NOUN'), ('شركة', 'NOUN'), ('اميركية', 'ADJ'), ('على', 'ADP'), ('رخصة', 'NOUN'), ('تصنيع', 'NOUN'), ('دبابة', 'NOUN'), ('"', 'PUNCT'), ('ليوبارد', 'X'), ('"', 'PUNCT'), ('الالمانية', 'ADJ')]


In [ ]:
#Dataset Class
class Dataset():

    ROOT = ('<root>', '<root>', 0)  # Pseudo-root

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            tmp = [Dataset.ROOT]
            for line in lines:
                if not line.startswith('#'):  # Skip lines with comments
                    line = line.rstrip()
                    if line:
                        columns = line.split('\t')
                        if columns[0].isdigit():  # Skip range tokens
                            tmp.append((columns[1], columns[3], int(columns[6])))
                    else:
                        yield tmp
                        tmp = [Dataset.ROOT]

In [ ]:
dep_train_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-train.conllu")
dep_dev_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-dev.conllu")
#Example sentence list
print(list(dep_train_english_data)[0])

dep_train_arab_data = Dataset("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-train.conllu")
dep_dev_arab_data = Dataset("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-dev.conllu")
print(list(dep_train_arab_data)[0])

dep_train_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-train.conllu")
dep_dev_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-dev.conllu")
print(list(dep_train_esp_data)[0])

dep_train_svk_data = Dataset("ud-treebanks-v2.5/UD_Swedish-LinES/sv_lines-ud-train.conllu")
dep_dev_svk_data = Dataset("ud-treebanks-v2.5/UD_Swedish-LinES/sv_lines-ud-dev.conllu")
print(list(dep_train_svk_data)[0])

[('<root>', '<root>', 0), ('Aesthetic', 'ADJ', 2), ('Appreciation', 'NOUN', 0), ('and', 'CCONJ', 5), ('Spanish', 'ADJ', 5), ('Art', 'NOUN', 2), (':', 'PUNCT', 2)]
[('<root>', '<root>', 0), ('برلين', 'X', 2), ('ترفض', 'VERB', 0), ('حصول', 'NOUN', 2), ('شركة', 'NOUN', 3), ('اميركية', 'ADJ', 4), ('على', 'ADP', 7), ('رخصة', 'NOUN', 3), ('تصنيع', 'NOUN', 7), ('دبابة', 'NOUN', 8), ('"', 'PUNCT', 11), ('ليوبارد', 'X', 9), ('"', 'PUNCT', 11), ('الالمانية', 'ADJ', 9)]
[('<root>', '<root>', 0), ('Además', 'ADV', 4), ('se', 'PRON', 4), ('le', 'PRON', 4), ('pediría', 'VERB', 0), ('a', 'ADP', 7), ('las', 'DET', 7), ('empresas', 'NOUN', 4), ('interesadas', 'ADJ', 7), ('en', 'ADP', 10), ('prestar', 'VERB', 8), ('el', 'DET', 12), ('servicio', 'NOUN', 10), ('que', 'SCONJ', 15), ('se', 'PRON', 15), ('hagan', 'VERB', 4), ('cargo', 'NOUN', 15), ('de', 'ADP', 19), ('la', 'DET', 19), ('señalización', 'NOUN', 16), ('y', 'CCONJ', 22), ('la', 'DET', 22), ('cartelería', 'NOUN', 19), ('que', 'SCONJ', 24), ('cont

In [ ]:
PAD = '<pad>'
UNK = '<unk>'
def make_vocabs_head_tagger(gold_data):
    words = {PAD:0, UNK:1}
    tags =  {PAD:0}

    word_i = 2
    tag_i = 1

    for sentence in gold_data:
      for word,tag, _ in sentence:
        if tag not in tags:
          tags[tag] = tag_i
          tag_i +=1
        if word not in words:
          words[word] = word_i
          word_i +=1

    return words, tags

In [ ]:
word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_english_data)

In [ ]:
#Arc Parser and oracle move implementation
class Parser(object):

    def predict(self, words, tags):
        raise NotImplementedError

class ArcStandardParser(Parser):
    #parser = ArcStandardParser()

    MOVES = tuple(range(3))

    SH, LA, RA = MOVES  # Parser moves are specified as integers.

    @staticmethod
    def initial_config(num_words):

        #the heads list will always be as long as the number of words in the sentence
        config = (0,[],[0]*num_words)
        return config

    @staticmethod
    def valid_moves(config):

        valid = []
        if config[0] < len(config[2]):
          valid.append(0)
        if len(config[1]) >= 2:
          valid.append(1)
          valid.append(2)
        return valid

    @staticmethod
    def next_config(config, move):

        #copy to avoid messing with the state of the current config
        new_config = copy.deepcopy(config)
        buffer = new_config[0]
        stack = new_config[1]
        heads = new_config[2]

        #SH, shift the buffer to the stack and add to the buffer
        if move == 0:

          stack.append(buffer)
          buffer +=1

        #LA, create an arc from the topmost to the second topmost words
        #in the stack. pop the second topmost from the stack
        elif move == 1:
          top_most = stack[-1]
          second_top_most = stack[-2]

          heads[second_top_most] = top_most
          stack.remove(second_top_most)

        #RA, create an arc from the second topmost to the topmost words
        #in the stack. pop the topmost from the stack
        elif move == 2:
          top_most = stack[-1]
          second_top_most = stack[-2]

          heads[top_most] = second_top_most
          stack.remove(top_most)


        next_conf=(buffer,stack, heads)


        return next_conf

    @staticmethod
    def is_final_config(config):
        # a config is always final if no more moves can be performed
        if not ArcStandardParser().valid_moves(config):
          return True



def oracle_moves(gold_heads):
    SH = 0
    LA = 1
    RA = 2

    moves, stack = [], []
    move, buffer = 0, 0
    moves_to_make = copy.copy(gold_heads[1:])
    heads = [0]*len(gold_heads)

    for _ in range((len(gold_heads)*2)-1):
      try:
        if len(stack)>=2 and gold_heads[stack[-2]] == stack[-1] and stack[-2] not in moves_to_make:

          yield (buffer,stack,heads), LA
          moves_to_make.remove(stack[-1])
          heads[stack[-2]] = stack[-1]
          stack.remove(stack[-2])

          moves.append(LA)
          move = LA

        elif len(stack)>=2 and gold_heads[stack[-1]] == stack[-2] and stack[-1] not in moves_to_make:

          yield (buffer,stack,heads), RA
          moves_to_make.remove(stack[-2])
          heads[stack[-1]] = stack[-2]
          stack.remove(stack[-1])

          moves.append(RA)
          move = RA

        else:
          yield (buffer,stack,heads), SH
          stack.append(buffer)
          buffer +=1
          moves.append(SH)
          move = SH
      except:
        pass

In [ ]:
#biLSTM concatenate words and tags (xi=e(wi)◦e(pi)) so there is no need to add ps2,1,0 here
def get_feats(config, word_ids):
  b, s, h = config

  vs2, vs1, vs0 = [0,0,0]
  b0 = 0
  if b < len(h):
    b0 = b

  if(len(s) == 1):
    vs0 = s[-1]

  elif(len(s) == 2):
    vs0 = s[-1]
    vs1 = s[-2]

  elif(len(s) >= 3):
    vs0 = s[-1]
    vs1 = s[-2]
    vs2 = s[-3]

  feats_from_config = [vs2, vs1, vs0, b0] #Index of vs2, vs1, vs0, bs0 from config
  feats_torch = torch.LongTensor(feats_from_config) #torch.Size([1, 4])
  return torch.LongTensor(feats_torch)

In [ ]:
def training_example(vocab_words, vocab_tags, gold_data):
    for sentence in gold_data: #[('<root>', '<root>', 0), ('Aesthetic', 'ADJ', 2), ('Appreciation', 'NOUN', 0), ('and', 'CCONJ', 5), ('Spanish', 'ADJ', 5), ('Art', 'NOUN', 2), (':', 'PUNCT', 2)]
        word_ids, tag_ids, gold_heads = [], [], []
        moves = []
        idxs = []
        #create id-lists for the current sentence
        for tup in (sentence):
            word_ids.append(vocab_words[tup[0]]) # Ids of ['<root>', 'Aesthetic'... ]
            tag_ids.append(vocab_tags[tup[1]]) # Ids of ['<root>', 'ADJ'... ]
            gold_heads.append(tup[2]) #[0, 2, ...]

        for config, move in oracle_moves(gold_heads): #for each sentence we get oracle moves
            idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
            idxs.append(idx_feats)
            moves.append(move)

        yield torch.LongTensor(word_ids).unsqueeze(0), torch.LongTensor(tag_ids).unsqueeze(0), torch.stack(idxs), torch.LongTensor(moves)

In [ ]:
#Example of one sentence in training example function
w_ids, t_ids, f_idx, mov = next(training_example(word_vocab, tag_vocab, dep_train_english_data))
print("word ids: ",w_ids)
print("tag ids: ",t_ids)
print("mlp features indexs: " ,f_idx.shape)
print("force trinning moves: ", mov.shape)

word ids:  tensor([[2, 3, 4, 5, 6, 7, 8]])
tag ids:  tensor([[1, 2, 3, 4, 2, 3, 5]])
mlp features indexs:  torch.Size([13, 4])
force trinning moves:  torch.Size([13])


In [ ]:
#"Encoder" Implementation
class BiLSTM(nn.Module):
    def __init__(self, num_words, num_pos, word_emb_dim = 100, pos_emb_dim=25, hidden_dim = 125, n_layers=2):
        super().__init__()
        # for the BiLSTM
        self.ew = nn.Embedding(num_words, word_emb_dim)
        self.ep = nn.Embedding(num_pos, pos_emb_dim)
        self.bi_lstm = nn.LSTM(word_emb_dim + pos_emb_dim, hidden_dim, num_layers = n_layers, batch_first=True, bidirectional = True)

    def forward(self, word, pos):
        ew = self.ew(word)            # Shape: [Batch_size, n_words, word_emb_dim]
        ep = self.ep(pos)             # Shape: [Batch_size, n_words, pos_emb_dim]
        x = torch.cat((ew, ep), 2)    # Shape: [Batch_size, n_words, word_emb_dim + pos_emb_dim]
        output, _ = self.bi_lstm(x)   # Shape: [Batch_size, n_words, 2*(word_emb_dim + pos_emb_dim)]

        return output

In [ ]:
#w_ids, t_ids, g_heads sentence example
model_lstm = BiLSTM(len(word_vocab), len(tag_vocab))
test_output = model_lstm.forward(w_ids, t_ids)
print("Shape BiLSTM feats: ",test_output.shape)

Shape BiLSTM feats:  torch.Size([1, 7, 250])


In [ ]:
class MLP(nn.Module):
    def __init__(self, word_emb_dim = 100, pos_emb_dim=25, hidden_unit = 100, output_dim = 3):
        super().__init__()
        self.hidden = nn.Linear(4*2*(word_emb_dim + pos_emb_dim), hidden_unit)
        self.relu = nn.ReLU() # try relu
        self.output = nn.Linear(hidden_unit, output_dim)

    def forward(self, features):
        hn = self.hidden(features)
        out = self.output(self.relu(hn))

        return out

In [ ]:
#Example of MLP
#feats_from_config is a list of list of index bc each list represent a sentence, in this case batch_size = 1 so [[ words ]]
#this is for no batch (we are using this one)
print("Example for no batch | biLSTM output shape: ", test_output.shape)
feats_from_config = [0, 3, 4, 5] #Index of vs2, vs1, vs0, bs0 from config
feats_torch = torch.LongTensor(feats_from_config) #torch.Size([1, 4])
test_input_mlp = test_output.index_select(1, feats_torch)
print("Shape input for mlp: " ,test_input_mlp.shape)
test_input_mlp = test_input_mlp.view(-1,test_input_mlp.size(1)*test_input_mlp.size(2))
print("Concat embeds shape: ", test_input_mlp.shape)

print("#################################")

#Batch size > 1 case
test_output2 = torch.cat((test_output,test_output), 0)
print("Example for batch feats | biLSTM output shape: ", test_output2.shape)
feats_from_config2 = [[0, 3, 4, 5], [0, 3, 4, 5]]
feats_torch2 = torch.LongTensor(feats_from_config2)
indexs = feats_torch2.unsqueeze(2).expand(feats_torch2.size(0), feats_torch2.size(1), test_output2.size(2))
test_input_mlp2 = torch.gather(test_output2, 1, indexs)
print("Shape input for mlp: " ,test_input_mlp2.shape)
test_input_mlp2 = test_input_mlp2.view(-1,test_input_mlp2.size(1)*test_input_mlp2.size(2))
print("Concat embeds shape: ",test_input_mlp2.shape)

print("################################")
#Case 3 (list of list of index for one sentence (batch=1))
#Every list of index represent the setting for a move, in this case 2
test_output3 = test_output
print("Example for no batch | biLSTM output shape: ", test_output3.shape)
feats_from_config3 = [[0, 3, 4, 5], [0, 3, 4, 5]]
feats_torch3 = torch.LongTensor(feats_from_config3) #torch.Size([2, 4])
test_input_mlp3 = [test_output3.index_select(1, idxs) for idxs in feats_torch3]
test_input_mlp3 = torch.stack(test_input_mlp3)
test_input_mlp3 = test_input_mlp3.squeeze(1)
print("Shape input for mlp: " ,test_input_mlp3.shape)
test_input_mlp3 = test_input_mlp3.view(-1,test_input_mlp3.size(1)*test_input_mlp3.size(2))
print("Concat embeds shape: ", test_input_mlp3.shape)

Example for no batch | biLSTM output shape:  torch.Size([1, 7, 250])
Shape input for mlp:  torch.Size([1, 4, 250])
Concat embeds shape:  torch.Size([1, 1000])
#################################
Example for batch feats | biLSTM output shape:  torch.Size([2, 7, 250])
Shape input for mlp:  torch.Size([2, 4, 250])
Concat embeds shape:  torch.Size([2, 1000])
################################
Example for no batch | biLSTM output shape:  torch.Size([1, 7, 250])
Shape input for mlp:  torch.Size([2, 4, 250])
Concat embeds shape:  torch.Size([2, 1000])


In [ ]:
# example data features = test_input_mlp2
model_MLP = MLP()
test_output_mlp = model_MLP.forward(test_input_mlp)
print("Shape BiLSTM feats: ",test_output_mlp.shape)
test_output_mlp2 = model_MLP.forward(test_input_mlp2)
print("Shape BiLSTM feats: ",test_output_mlp2.shape)

Shape BiLSTM feats:  torch.Size([1, 3])
Shape BiLSTM feats:  torch.Size([2, 3])


In [ ]:
class Blackbox(nn.Module): #need to change name lol haha
    def __init__(self, vocab_words, vocab_tags, num_words, num_pos):
        super().__init__()

        self.model_lstm = BiLSTM(num_words, num_pos)
        self.model_MLP = MLP()
        self.vocab_words = vocab_words
        self.vocab_tags = vocab_tags

    def forward(self, word_ids, pos_ids, idxs_feats):
        lstm_output = self.model_lstm.forward(word_ids, pos_ids) #biLSTM output shape:  torch.Size([1, 7, 250])
        #input_mlp = lstm_output.index_select(1, idxs_feats) #Shape input for mlp:  torch.Size([1, 4, 250])
        input_mlp = [lstm_output.index_select(1, idxs) for idxs in idxs_feats] #len(n) = moves
        input_mlp = torch.stack(input_mlp) #to tensor
        input_mlp = input_mlp.squeeze(1) #shape tesnro ([n, 4, 250]) where n: moves
        input_mlp = input_mlp.view(-1,input_mlp.size(1)*input_mlp.size(2)) #Concat embeds shape:  torch.Size([n, 1000])
        output_mlp = self.model_MLP.forward(input_mlp) #torch.Size([n, 3])

        return output_mlp

    def predict(self, words, tags):
        for idx, (w, t) in enumerate(zip(words,tags)): #words string to words ids
            if w in self.vocab_words:
                words[idx] = self.vocab_words[w]
            else:
                words[idx] = self.vocab_words['<unk>']
            if t in self.vocab_tags:
                tags[idx] = self.vocab_tags[t]
            else:
                tags[idx] = self.vocab_tags['<pad>']

        config = ArcStandardParser.initial_config(len(words))
        while not ArcStandardParser.is_final_config(config):
            idx_feats = get_feats(config, words) #take features #idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
            valid_moves = ArcStandardParser.valid_moves(config)

            with torch.no_grad():
                output_move = torch.argmax(self.forward(words, tags, idx_feats)) #output_mlp = self.forward(words, tags, idx_feats)
            if(output_move in valid_moves):
                config = parser.next_config(config, output_move)
            else:
                config = parser.next_config(config, valid_moves[-1])

        return config[2]

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import tqdm
torch.autograd.set_detect_anomaly(True)

def train_model(train_data, dev_data, n_epochs = 5, lim_ep = 4000, batch_size = 0, lr=1e-3):
    word_vocab, tag_vocab = make_vocabs_head_tagger(train_data)
    model = Blackbox(word_vocab, tag_vocab, len(word_vocab), len(tag_vocab)) #  model_lstm = BiLSTM(len(word_vocab), len(tag_vocab)) |  model_MLP = MLP()
    #model.to(device)
    # Initialize the optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    dev_losses = []
    dev_accuracies = []
    info = {'dev loss': 0, 'dev acc': 0}

    x = 0
    N_tqdm = len(list(train_data))

    with tqdm.tqdm(total=n_epochs * N_tqdm) as pbar: #len(dep_train_english_data) = 3753
        for t in range(n_epochs):
            x = 0
            model.train()
            # start training
            running_loss = 0
            t_data = training_example(word_vocab, tag_vocab, train_data)

            for w_ids, t_ids, f_idx, move in t_data: #This represet a sentence w_ids, t_ids, f_idx, mov
                pbar.set_description(f'Sentence {x+1}')
                optimizer.zero_grad()
                output_mlp = model.forward(w_ids, t_ids, f_idx)
                loss = F.cross_entropy(output_mlp, move)
                loss.backward() #retain_graph=True
                optimizer.step()


                running_loss += loss.item()
                x += 1
                if(x%250 == 0):
                    print("Loss: ", running_loss / x)


                pbar.update(1)

                train_losses.append(running_loss / x)

                if(x == lim_ep): #Train only lim_ep sentences
                  break

            UAS_t = uas_tst(model, word_vocab, tag_vocab, dev_data)
            print("Epoch: %s | AUS: %.2f" %(t, UAS_t * 100), "%")

    return model

In [ ]:
parser = ArcStandardParser()

In [ ]:
dep_train_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSD/zh_gsd-ud-train.conllu") #3176
print(len(list(dep_train_cn_data)))
print(list(dep_train_cn_data)[0])
dep_train_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSDSimp/zh_gsdsimp-ud-train.conllu") #3176
print(len(list(dep_train_cn_data)))
print(list(dep_train_cn_data)[0])

3997
[('<root>', '<root>', 0), ('看似', 'AUX', 2), ('簡單', 'ADJ', 5), ('，', 'PUNCT', 5), ('只', 'ADV', 5), ('是', 'VERB', 0), ('二', 'NUM', 7), ('選', 'VERB', 9), ('一', 'NUM', 7), ('做', 'VERB', 5), ('決擇', 'NOUN', 9), ('，', 'PUNCT', 5), ('但', 'ADV', 22), ('其實', 'ADV', 22), ('他們', 'PRON', 15), ('代表', 'VERB', 22), ('的', 'PART', 15), ('是', 'AUX', 22), ('你', 'PRON', 19), ('周遭', 'NOUN', 22), ('的', 'PART', 19), ('親朋', 'NOUN', 22), ('好友', 'NOUN', 5), ('，', 'PUNCT', 5), ('試', 'VERB', 5), ('著', 'PART', 24), ('給', 'VERB', 24), ('你', 'PRON', 26), ('不同', 'ADJ', 30), ('的', 'PART', 28), ('意見', 'NOUN', 26), ('，', 'PUNCT', 5), ('但', 'ADV', 39), ('追根究底', 'VERB', 39), ('，', 'PUNCT', 39), ('最後', 'NOUN', 36), ('決定', 'VERB', 39), ('的', 'PART', 36), ('還是', 'AUX', 39), ('自己', 'PRON', 5), ('。', 'PUNCT', 5)]
3997
[('<root>', '<root>', 0), ('看似', 'AUX', 2), ('简单', 'ADJ', 5), ('，', 'PUNCT', 5), ('只', 'ADV', 5), ('是', 'VERB', 0), ('二', 'NUM', 7), ('选', 'VERB', 9), ('一', 'NUM', 7), ('做', 'VERB', 5), ('决择', 'NOUN', 9), ('，

In [ ]:
dep_train_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSDSimp/zh_gsdsimp-ud-train.conllu") #3997
dep_dev_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSDSimp/zh_gsdsimp-ud-dev.conllu")
model_BB = train_model(dep_train_cn_data, dep_dev_cn_data) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_svk_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_svk_data))) #71% |(1e-3)78.82 (2000) |

Sentence 251:   1%|▏         | 250/19985 [01:59<1:57:16,  2.80it/s]

Loss:  0.5553316417336464


Sentence 501:   3%|▎         | 500/19985 [03:46<2:37:08,  2.07it/s]

Loss:  0.4307987672071904


Sentence 751:   4%|▍         | 750/19985 [05:46<2:49:16,  1.89it/s]

...

Loss:  0.04040570567264831


Sentence 3751:  99%|█████████▉| 19738/19985 [2:47:02<02:17,  1.80it/s]

Loss:  0.040488850576418496


Sentence 3997: 100%|██████████| 19985/19985 [2:51:09<00:00,  1.95it/s]

Epoch: 4 | AUS: 76.86 %


In [ ]:
dep_train_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSD/zh_gsd-ud-train.conllu") #3997
dep_dev_cn_data = Dataset("ud-treebanks-v2.5/UD_Chinese-GSD/zh_gsd-ud-dev.conllu")
model_BB = train_model(dep_train_cn_data, dep_dev_cn_data) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_svk_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_svk_data))) #71% |(1e-3)78.82 (2000) |

Sentence 251:   1%|▏         | 250/19985 [02:01<1:57:37,  2.80it/s]

Loss:  0.5547375439405441


Sentence 501:   3%|▎         | 500/19985 [03:49<2:36:23,  2.08it/s]

Loss:  0.4341847336702049


Sentence 751:   4%|▍         | 750/19985 [05:49<2:52:32,  1.86it/s]

...

Loss:  0.042714012193468466


Sentence 3751:  99%|█████████▉| 19738/19985 [2:48:11<02:21,  1.75it/s]

Loss:  0.0425251116497219


Sentence 3997: 100%|██████████| 19985/19985 [2:52:21<00:00,  1.93it/s]

Epoch: 4 | AUS: 76.20 %


In [ ]:
dep_train_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-train.conllu") #3753 sentences
dep_dev_english_data = Dataset("ud-treebanks-v2.5/UD_English-GUM/en_gum-ud-dev.conllu")
model_BB = train_model(dep_train_english_data, dep_dev_english_data) #UAS: 0.71 | now: 72.95 | 76.66 (lr -3) |81.50% (2x2000)
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_english_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_english_data)))

Sentence 251:   1%|▏         | 250/18765 [02:07<1:47:02,  2.88it/s]

Loss:  0.47540055483579635


Sentence 501:   3%|▎         | 500/18765 [04:22<2:18:45,  2.19it/s]

Loss:  0.37458727664966135


Sentence 751:   4%|▍         | 750/18765 [06:05<1:49:15,  2.75it/s]

...

Loss:  0.036555856259566206


Sentence 3751: 100%|█████████▉| 18762/18765 [2:17:45<00:01,  2.67it/s]

Loss:  0.03566186669803062


Sentence 3753: 100%|██████████| 18765/18765 [2:20:22<00:00,  2.23it/s]

Epoch: 4 | AUS: 82.66 %


In [ ]:
dep_train_arab_data = Dataset("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-train.conllu") #6075 senteces
dep_dev_arab_data = Dataset("ud-treebanks-v2.5/UD_Arabic-PADT/ar_padt-ud-dev.conllu")
model_BB = train_model(dep_train_arab_data, dep_dev_arab_data) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_arab_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_arab_data))) #70.62% | now: 73.58% |(1e-3) 74.35 (2000) | 79.36%(16150)

Sentence 2:   0%|          | 1/4000 [00:00<21:02,  3.17it/s]

Loss: 1.0983740091323853


Sentence 252:   6%|▋         | 251/4000 [02:51<35:39,  1.75it/s]

Loss: 0.25778454542160034


Sentence 502:  13%|█▎        | 501/4000 [05:42<41:46,  1.40it/s]

...

Loss: 9.466408664593473e-05


Sentence 16002: : 16001it [3:16:35,  1.84it/s]

Loss: 0.06886310130357742


Sentence 16150: : 16150it [3:18:04,  1.36it/s]


0.7936


In [ ]:
dep_train_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-train.conllu") #14187
dep_dev_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-dev.conllu")
model_BB = train_model(dep_train_esp_data, dep_dev_esp_data) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_esp_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_esp_data))) #72% | now: 80.11% | 81.26% (2000) |

Sentence 251:   0%|          | 250/70935 [02:46<15:16:00,  1.29it/s]

Loss:  0.44064745987951753


Sentence 501:   1%|          | 500/70935 [05:27<10:59:14,  1.78it/s]

Loss:  0.3461833526622504


Sentence 751:   1%|          | 750/70935 [07:56<13:32:08,  1.44it/s]

...

Loss:  0.039293514705632575


Sentence 4000:  28%|██▊       | 20000/70935 [4:00:21<10:45:47,  1.31it/s]

Loss:  0.03866852368081392


Sentence 4000:  28%|██▊       | 20000/70935 [4:07:53<10:31:20,  1.34it/s]

Epoch: 4 | AUS: 86.57 %


In [ ]:
dep_train_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-train.conllu") #14187
dep_dev_esp_data = Dataset("ud-treebanks-v2.5/UD_Spanish-GSD/es_gsd-ud-dev.conllu")
model_BB = train_model(dep_train_esp_data, dep_dev_esp_data, n_epochs = 2, lim_ep=10000) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_esp_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_esp_data))) #72% | now: 80.11% | 81.26% (2000) |

Sentence 251:   1%|          | 250/28374 [02:22<5:17:36,  1.48it/s]

Loss:  0.442774000748992


Sentence 501:   2%|▏         | 500/28374 [04:41<3:39:17,  2.12it/s]

Loss:  0.3450293645914644


Sentence 751:   3%|▎         | 750/28374 [06:52<4:37:14,  1.66it/s]

...

Loss:  0.1012933226662773


Sentence 10000:  70%|███████   | 20000/28374 [3:17:23<1:33:16,  1.50it/s]

Loss:  0.10114526392016596


Sentence 10000:  70%|███████   | 20000/28374 [3:24:13<1:25:30,  1.63it/s]

Epoch: 1 | AUS: 88.18 %


In [ ]:
dep_train_svk_data = Dataset("ud-treebanks-v2.5/UD_Swedish-LinES/sv_lines-ud-train.conllu") #3176
dep_dev_svk_data = Dataset("ud-treebanks-v2.5/UD_Swedish-LinES/sv_lines-ud-dev.conllu")
model_BB = train_model(dep_train_svk_data, dep_dev_svk_data) #UAS:
#word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_svk_data)
#print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_svk_data))) #71% |(1e-3)78.82 (2000) |

Sentence 252:   2%|▏         | 251/15880 [01:19<1:32:41,  2.81it/s]

Loss:  0.44293348531611265


Sentence 501:   3%|▎         | 500/15880 [02:52<1:31:26,  2.80it/s]

Loss:  0.3616340642780997


Sentence 751:   5%|▍         | 750/15880 [04:26<1:09:14,  3.64it/s]

...

Loss:  0.03917843671839811


Sentence 3001:  99%|█████████▉| 15704/15880 [1:52:10<01:03,  2.76it/s]

Loss:  0.03915995189891199


Sentence 3176: 100%|██████████| 15880/15880 [1:56:16<00:00,  2.28it/s]

Epoch: 4 | AUS: 82.74 %


In [ ]:
def uas_tst(modelo, vocab_words, vocab_tags, gold_data):
    correct, total = 0, 0
    for sentence in gold_data:
      words,tags, heads = [], [], []

      for word, tag, head in (sentence):
        words.append(word)
        tags.append(tag)
        heads.append(head)
      #print(words)
      preds = predict_tst(modelo, vocab_words, vocab_tags, words, tags)
      #print(preds)

      for i in range(1,len(preds)):
        if heads[i] == preds[i]:
          correct +=1
        total +=1

    return correct/total

def predict_tst(modelo, vocab_words, vocab_tags, words, tags):
    for idx, (w, t) in enumerate(zip(words,tags)): #words string to words ids
      if w in vocab_words:
        words[idx] = vocab_words[w]
      else:
        words[idx] = vocab_words['<unk>']
      if t in vocab_tags:
        tags[idx] =vocab_tags[t]
      else:
        tags[idx] = vocab_tags['<pad>']


    words = torch.LongTensor(words).unsqueeze(0)
    tags = torch.LongTensor(tags).unsqueeze(0)

    parser = ArcStandardParser()
    (SH, LA, RA) = parser.MOVES
    config = parser.initial_config(words.shape[1])
    while not parser.is_final_config(config):
      idx_feats = get_feats(config, words) #take features #idx_feats = get_feats(config, word_ids) #long_tensor([vs0, vs1, vs2, b0]))
      #idx_feats = get_feats(config, words) #long_tensor([vs0, vs1, vs2, b0]))
      idx_feats = idx_feats.unsqueeze(0)
      valid_moves = parser.valid_moves(config)

      with torch.no_grad():
        output_move = torch.argmax(modelo.forward(words, tags, idx_feats)) #output_mlp = self.forward(words, tags, idx_feats)
      if(output_move in valid_moves):
        config = parser.next_config(config, output_move)
      else:
        config = parser.next_config(config, valid_moves[-1])

    return config[2]

In [ ]:
dep_train_arabic_data = Dataset("projectivized_arabic_train.conllu")
dep_dev_arabic_data = Dataset("projectivized_arabic_dev.conllu")
word_vocab, tag_vocab = make_vocabs_head_tagger(dep_train_arabic_data)
model_BB = train_model(dep_train_arabic_data) #UAS:
print('{:.4f}'.format(uas_tst(model_BB, word_vocab, tag_vocab, dep_dev_arabic_data))) #71%